<div style="width:1000 px">

<div style="float:right; width:98 px; height:98px;">
<img src="https://raw.githubusercontent.com/Unidata/MetPy/master/metpy/plots/_static/unidata_150x150.png" alt="Unidata Logo" style="height: 98px;">
</div>

<h1>Primer</h1>
<h3>Unidata Python Workshop</h3>

<div style="clear:both"></div>
</div>

<hr style="height:2px;">

<div style="float:right; width:250 px"><img src="http://www.contribute.geeksforgeeks.org/wp-content/uploads/numpy-logo1.jpg" alt="NumPy Logo" style="height: 250px;"></div>


## Overview:

* **Teaching:** 15 minutes
* **Exercises:** 15 minutes

### Questions
1. What are arrays?
2. How can arrays be manipulated effectively in Python?
3. How do we work with unit information with MetPy?

### Objectives
1. Slice and index the array
2. Perform a meteorological calculation on an array of data using MetPy.
3. Use advanced methods to index into arrays

### Resources
- [MetPy Documentation](https://unidata.github.io/MetPy/)
- [MetPy Calculation List](https://unidata.github.io/MetPy/api/generated/metpy.calc.html)
- [MetPy Constants](https://unidata.github.io/MetPy/api/generated/metpy.constants.html)

## 1. Index and slice arrays

Indexing is how we pull individual data items out of an array. Slicing extends this process to pulling out a regular set of the items.

In [ ]:
# Convention for import to get shortened namespace
import numpy as np

In [ ]:
# Create an array for testing
a = np.arange(12).reshape(3, 4)

In [ ]:
a

Indexing in Python is 0-based, so the command below looks for the 2nd item along the first dimension (row) and the 3rd along the second dimension (column).

In [ ]:
a[1, 2]

Can also just index on one dimension

In [ ]:
a[2]

Negative indices are also allowed, which permit indexing relative to the end of the array.

In [ ]:
a[0, -1]

Slicing syntax is written as `start:stop[:step]`, where all numbers are optional.
- defaults: 
  - start = 0
  - end = len(dim)
  - step = 1
- The second colon is also optional if no step is used.

It should be noted that end represents one past the last item; one can also think of it as a half open interval: `[start, end)`

In [ ]:
# Get the 2nd and 3rd rows
a[1:3]

In [ ]:
# All rows and 3rd column
a[:, 2]

In [ ]:
# ... can be used to replace one or more full slices
a[..., 2]

In [ ]:
# Slice every other row
a[::2]

In [ ]:
# Slice out every other column
a[:, ::2]

In [ ]:
# Slice every other item along each dimension -- how would we do this


## 2. Use units from MetPy with calculations

In [ ]:
# Import MetPy's units registry
from metpy.units import units

In [ ]:
length = 8 * units.feet
print(length * length)

In [ ]:
distance = 10 * units.mile
time = 15 * units.minute
avg_speed = distance / time
print(avg_speed)
print(avg_speed.to_base_units())
print(avg_speed.to('mph'))

## 3. Perform meteorological calculations using MetPy

In [ ]:
np.random.seed(19990503)  # So we all have the same data
u = np.random.randint(0, 45, 10) * units('m/s')
v = np.random.randint(0, 45, 10) * units('m/s')

In [ ]:
print(u)
print(v)

In [ ]:
import metpy.calc as mpcalc

speed = mpcalc.wind_speed(u, v)
direction = mpcalc.wind_direction(u, v)

In [ ]:
print(speed)
print(np.rad2deg(direction))

In [ ]:
print(np.mean(speed))

In [ ]:
print(np.mean(np.rad2deg(direction)))
print(np.std(np.rad2deg(direction)))

Let's use MetPy to calculate the dewpoint from the current temperature and relative humidity:

In [ ]:
import metpy.calc as mpcalc
mpcalc.dewpoint_rh(25 * units.degC, 0.75)

Thanks to units, this can work with Fahrenheit as well:

In [ ]:
td = mpcalc.dewpoint_rh(77 * units.degF, 0.75)
td

And you can get it back in Fahrenheit as:

In [ ]:
td.to('degF')

MetPy also has a library of useful constants, similar to those in `scipy.constants`, that are important for meteorology and have appropriate dimensionality included:

In [ ]:
import metpy.constants as consts

We can look at the docstring for the module (or go to the web documentation) to see a list of the available contants:

In [ ]:
consts?

So for the density of liquid water (nominally at 0C), we can use:

In [ ]:
consts.density_water

Or for a more symbolic and shorter notation, you can also use:

In [ ]:
consts.rho_l

### Exercise #1

## 4. Advanced NumPy Indexing
### Boolean indexing
Numpy can easily create arrays of boolean values and use those to select certain values to extract from an array

In [ ]:
# Create some synthetic data representing temperature and wind speed data
np.random.seed(19990503)  # Make sure we all have the same data
temp = (20 * np.cos(np.linspace(0, 2 * np.pi, 100)) +
        50 + 2 * np.random.randn(100)) * units.degC
spd = (np.abs(10 * np.sin(np.linspace(0, 2 * np.pi, 100)) +
              10 + 5 * np.random.randn(100))) * units('m/s')

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.plot(temp.m, 'tab:red')
plt.plot(spd.m, 'tab:blue');

By doing a comparision between a NumPy array and a value, we get an
array of values representing the results of the comparison between
each element and the value

In [ ]:
temp > 45 * units.degC

We can take the resulting array and use this to index into the
NumPy array and retrieve the values where the result was true

In [ ]:
print(temp[temp > 45 * units.degC])

So long as the size of the boolean array matches the data, the boolean array can come from anywhere

In [ ]:
print(temp[spd > 10 * units('m/s')])

In [ ]:
# Make a copy so we don't modify the original data
temp2 = temp.copy()

# Replace all places where spd is <10 with NaN (not a number) so matplotlib skips it
temp2[spd < 10 * units('m/s')] = np.nan * units.degC
plt.plot(temp2, 'tab:red')

Can also combine multiple boolean arrays using the syntax for bitwise operations. **MUST HAVE PARENTHESES** due to operator precedence.

In [ ]:
print(temp[(temp < 45 * units.degC) & (spd > 10 * units('m/s'))])

### Arrays of indices

You can also use a list or array of indices to extract particular values--this is a natural extension of the regular indexing. For instance, just as we can select the first element:

In [ ]:
print(temp[0])

We can also extract the first, fifth, and tenth elements:

In [ ]:
print(temp[[0, 4, 9]])

One of the ways this comes into play is trying to sort numpy arrays using `argsort`. This function returns the indices of the array that give the items in sorted order. So for our temp "data":

In [ ]:
inds = np.argsort(temp)
print(inds)

We can use this array of indices to pass into temp to get it in sorted order:

In [ ]:
print(temp[inds])

Or we can slice `inds` to only give the 10 highest temperatures:

In [ ]:
ten_highest = inds[-10:]
print(temp[ten_highest])

There are other numpy arg functions that return indices for operating:

In [ ]:
np.*arg*?

### Exercise #2